In [1]:
import numpy as np
import matplotlib.pyplot as plt


# 解析 TSP 文件
def parse_tsp(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        cities = []
        for line in lines:
            parts = line.split()
            if len(parts) == 3 and parts[0].isdigit():
                cities.append((float(parts[1]), float(parts[2])))
        return cities

# 计算两点间距离
def distance(city1, city2):
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# 计算路径长度
def path_length(cities, tour):
    total_distance = 0
    for i in range(len(tour)):
        total_distance += distance(cities[tour[i-1]], cities[tour[i]])
    return total_distance

# 解析 .opt.tour 文件，以比较最优路径长度
def parse_opt_tour(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        opt_tour = []
        for line in lines:
            parts = line.split()
            if parts[0].isdigit():
                opt_tour.append(int(parts[0]) - 1)  # 减1是因为TSP文件通常从1开始编号，而Python列表从0开始
        return opt_tour

# 根据 .opt.tour 文件计算路径长度
def calculate_opt_length(cities, opt_tour):
    total_distance = 0
    for i in range(len(opt_tour)):
        total_distance += distance(cities[opt_tour[i - 1]], cities[opt_tour[i]])
    return total_distance

# 解析 eli51.tsp 和 eli51.opt.tour 文件
cities = parse_tsp('eil51.tsp.txt')
opt_tour = parse_opt_tour('eil51.opt.tour.txt')
opt_length = calculate_opt_length(cities, opt_tour)


In [2]:
# 2-opt 算法
def two_opt_swap(tour, i, j):
    new_tour = list(tour[:i])
    new_tour.extend(reversed(tour[i:j + 1]))
    new_tour.extend(tour[j + 1:])
    return np.array(new_tour)

# 移动单个城市的算法
def move_city(tour, city_index, new_position):
    city = tour[city_index]
    new_tour = np.delete(tour, city_index)  # 删除原位置的城市
    new_tour = np.insert(new_tour, new_position, city)  # 在新位置插入城市
    return new_tour

In [3]:
# 不同的冷却计划
def linear_cooling(current_temp, alpha):
    return current_temp - alpha

def exponential_cooling(current_temp, alpha):
    return current_temp * alpha

def logarithmic_cooling(current_temp, alpha, iteration):
    return current_temp / (1 + alpha * np.log(1 + iteration))

In [ ]:
# 2-opt策略的模拟退火算法
def simulated_annealing_2_opt(cities, initial_temp, final_temp, alpha, cooling_schedule):
    current_temp = initial_temp
    current_tour = np.random.permutation(len(cities))
    current_length = path_length(cities, current_tour)
    #iteration = 1
    
    while current_temp > final_temp:
        i, j = np.sort(np.random.choice(len(cities), 2, replace=False))
        new_tour = two_opt_swap(current_tour, i, j)
        new_length = path_length(cities, new_tour)
        
        if new_length < current_length or np.random.rand() < np.exp((current_length - new_length) / current_temp):
            current_tour = new_tour
            current_length = new_length
            
        #current_temp = cooling_schedule(current_temp, alpha, iteration)
       # iteration += 1
        current_temp *= alpha #默认使用exponential
    
    return current_tour, current_length


# 移动城市策略的模拟退火算法
def simulated_annealing_move_city(cities, initial_temp, final_temp, alpha, cooling_schedule):
    current_temp = initial_temp
    current_tour = np.random.permutation(len(cities))
    current_length = path_length(cities, current_tour)
   # iteration = 1
    
    while current_temp > final_temp:
        city_index = np.random.randint(len(cities))
        new_position = np.random.randint(len(cities))
        new_tour = move_city(current_tour, city_index, new_position)
        new_length = path_length(cities, new_tour)
        
        if new_length < current_length or np.random.rand() < np.exp((current_length - new_length) / current_temp):
            current_tour = new_tour
            current_length = new_length
            
        # current_temp = cooling_schedule(current_temp, alpha, iteration)
        #iteration += 1
        current_temp *= alpha #默认使用exponential
    
    return current_tour, current_length